<a href="https://colab.research.google.com/github/axel-sirota/practical-nlp/blob/main/3-text-generation/Practical_NLP_10_RentalGenerator_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Oct 15 23:07:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install textblob 'gensim==4.2.0' 'keras-nlp' 'transformers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
from transformers import GPT2Tokenizer, TFGPT2Model
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=25
buffer_size = 64
corpus_size=25000
test_corpus_size=5000
# Batch size
batch_size = 64
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%%writefile get_data.sh
if [ ! -f train_corpus_descriptions_airbnb.csv ]; then
  wget -O train_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
fi

if [ ! -f test_corpus_descriptions_airbnb.csv ]; then
    wget -O test_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/a29bbkg8hi4q4f4/test_corpus_descriptions_airbnb.csv?dl=0
fi

Overwriting get_data.sh


In [5]:
!bash get_data.sh

In [6]:
train_path = "./train_corpus_descriptions_airbnb.csv"
test_path = "./test_corpus_descriptions_airbnb.csv"
# Read, then decode for py2 compat.
airbnb_reviews = pd.read_csv(train_path, header=None, names=["review"]).dropna().sample(n=corpus_size).reset_index(drop=True)
test_airbnb_reviews = pd.read_csv(test_path, header=None, names=["review"]).dropna().sample(n=test_corpus_size).reset_index(drop=True)


In [7]:
airbnb_reviews.head()

,review
0,"Comfortable, quiet garden-facing double room w..."
1,Welcome to the Giant Oaks House located just o...
2,The apartment is located just 2 minutes walk f...
3,Stay for 90+ nights (minimum nights and rates ...
4,"Ideal place for tourists , visiting los angele..."


In [8]:
def preprocess_text(text, should_join=True):
    text = str(text)
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [9]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(preprocess_text(row.review, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(airbnb_reviews)


In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_len_single_sentence=maximum, max_len=maximum)
gpt_model = TFGPT2Model.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [11]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


1

In [12]:
vocab_size = len(tokenizer.get_vocab())

In [13]:
tokenizer(preprocess_text(airbnb_reviews.review[1], should_join=True), return_tensors='tf', padding=True)

{'input_ids': <tf.Tensor: shape=(1, 80), dtype=int32, numpy=
array([[   86,  9571,   284,   262,  6175,   267,  4730,  2156,  5140,
          655,   572,  5366,  8681,   257,   303,   428,  3095,  4289,
         1363,  3033, 10017, 18570, 22563,   276, 45926,   290,   736,
        33179,  1049,   329, 28175,   981, 34603,  1263,  4950,   614,
         1468, 26210,  5509,   691,  2431,   284,  9436,  1976,   346,
         6122,  3952,   290,   262,  9003,  5366,  8681, 10283,   318,
          655,  2431,  1497,  1748,   286, 38132,   259,  4433,  1790,
         3381, 14447,  5964,   284,  2583, 10650,   329,   739,  1528,
          523,  2045,   329, 48028,   286,   530,  1227, 30274]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 80), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
def get_ids_tensor(df):
  all_ids = tf.constant(np.zeros((1, maximum)), dtype='int32')
  for review in df.review:
      review = preprocess_text(review, should_join=True)
      value = tokenizer(review, return_tensors='tf', padding=True)['input_ids']
      if value.shape[1] < maximum:
        value = tf.concat([tf.cast(value, tf.int32), tf.zeros((1, maximum - value.shape[1]), dtype='int32')], axis=1)
      value = value[:,:maximum]
      value = tf.reshape(value, [1, maximum])
      output = tf.concat([all_ids,value], axis=0)
      all_ids = tf.reshape(output, [-1, maximum])
  return all_ids[1:]

all_ids = get_ids_tensor(df=airbnb_reviews)
print(all_ids)

In [ ]:
test_all_ids = get_ids_tensor(df=test_airbnb_reviews)

In [ ]:
#Prepare the dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_all_ids)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = ids_dataset.map(split_input_target)
test_dataset = test_ids_dataset.map(split_input_target)


In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size=batch_size, drop_remainder=True)
)
test_dataset = (
    test_dataset
    .batch(batch_size=batch_size, drop_remainder=True)
)

In [ ]:
dataset.take(1)

In [ ]:
class RentalGenerator(tf.keras.Model):
  def __init__(self, model, vocab_size, rnn_units):
    super().__init__(self)
    self.pretrained_layer = model
    self.rnn = tf.keras.layers.LSTM(rnn_units,
                                   activation='tanh',
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.pretrained_layer(x, training=False)[0]
    if states is None:
      states = self.rnn.get_initial_state(x)
    output = self.rnn(x, initial_state=states, training=training)
    x = output[0]
    states = output[1:]
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = RentalGenerator(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=vocab_size,
    model=gpt_model,
    rnn_units=rnn_units)
model.layers[0].trainable = False

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[perplexity])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, min_delta=0.03)
history = model.fit(dataset, epochs=epochs, validation_data=test_dataset, workers=5, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], val_metric=history.history['val_loss'], metric_name="Loss", title="Loss", ylim=5.0)

In [ ]:
plot_metrics(history.history['perplexity'], val_metric=history.history['val_perplexity'], metric_name="perplexity", title="perplexity", ylim=2000.0)

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, tokenizer, ix_to_word, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.tokenizer = tokenizer
    self.ix_to_word = ix_to_word
    
  def expand_dims_if_neccesary(self, input):
    if len(input.shape) < 3:
      input = tf.expand_dims(input, axis=0)
    return input

  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_words = preprocess_text(inputs, should_join=True)
    input_ids = self.tokenizer(input_words)
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    pred_id = predicted_ids.numpy()[0][0]
    # Convert from token ids to characters
    predicted_word = self.expand_dims_if_neccesary(tf.constant(self.ix_to_word[pred_id]))

    # Return the characters and model state.
    return predicted_word, states

In [ ]:
word_to_ix = tokenizer.get_vocab()
ix_to_word = {ix: word for word, ix in word_to_ix.items()}
print(ix_to_word)

In [ ]:
one_step_model = OneStep(model, tokenizer=lambda x: tokenizer(x, return_tensors='tf', padding=True)['input_ids'], ix_to_word=ix_to_word)
start = time.time()
states = None
description = tf.constant(['Midtown Sunny 2-Bedroom'])

for n in range(200):
  next_word, states = one_step_model.generate_one_step(description, states=states)
  description = tf.concat([description, next_word], axis=0)


result = tf.strings.join(description, separator=" ")
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)



In [ ]:
tf.saved_model.save(one_step_model, 'lstm_rental_generator')